<a href="https://colab.research.google.com/github/ZacharyMalonjao/FinMark-Machine-Learning/blob/main/Data_Cleaning_and_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
from scipy import stats

In [42]:
#STEP 1: Grab files from Github

demographics = pd.read_csv("https://raw.githubusercontent.com/ZacharyMalonjao/FinMark-Machine-Learning/refs/heads/main/data/demographics.csv")
transactions = pd.read_csv("https://raw.githubusercontent.com/ZacharyMalonjao/FinMark-Machine-Learning/refs/heads/main/data/transactions.csv")
interactions = pd.read_csv("https://raw.githubusercontent.com/ZacharyMalonjao/FinMark-Machine-Learning/refs/heads/main/data/interactions.csv")


In [43]:
#STEP 2 : Drop Duplicates
print("Old code duplicates:")
print(demographics.duplicated().sum())
print(interactions.duplicated().sum())
print(transactions.duplicated().sum())

demographics = demographics.drop_duplicates()
interactions = interactions.drop_duplicates()
transactions =transactions.drop_duplicates()

print("new code duplicates:")
print(demographics.duplicated().sum())
print(interactions.duplicated().sum())
print(transactions.duplicated().sum())


Old code duplicates:
177
180
185
new code duplicates:
0
0
0


In [44]:
#STEP 3a: Merge Demographics and Interactions
transaction_interaction = pd.merge(transactions, interactions, on = "CustomerID", how ="inner")
transaction_interaction.head()


,CustomerID,TransactionID,TransactionDate,Amount,ProductCategory,PaymentMethod,InteractionID,InteractionDate,Platform,InteractionType,Sentiment
0,9223891b-73ff-4d5c-b8ae-13ece82ee28b,f79588dd-3db9-4ffa-97f8-7de0e64259f1,2022-09-23,563.99,Clothing,Debit Card,df9b66ed-008c-4484-9e9a-83ab83206287,2024-06-29,Twitter,Like,Neutral
1,9243eebc-938f-480c-8564-16d503d250de,401c0fc9-60df-4455-ad78-67c132f9897d,2024-04-15,254.44,Automotive,PayPal,dcbde36b-7cfb-4220-93da-86427662206f,2023-10-18,Instagram,Share,Neutral
2,6e3e8eb8-bc0f-4ffe-9f74-5d5efec9502f,2034aebc-8280-4254-a667-92bcd1c2be4f,2024-06-03,590.52,Home & Garden,Bank Transfer,24017054-74d6-4579-abc2-0665046f6dc0,2024-05-13,Facebook,Share,Negative
3,958e5c8d-48ca-42dd-bb71-a766a374233a,fb24e098-3ab9-40a2-bcc3-b8ebb23f549a,2023-03-10,399.7,Home & Garden,PayPal,e8add018-5702-4fa1-b755-5aad0514df8b,12/06/2024,Instagram,Comment,Positive
4,39c6e7d2-6c4b-44c0-8961-ddc1ecbdb0c6,833b026a-7c02-4101-832d-62c07569b0f6,2024-01-26,296.99,Clothing,PayPal,8ecb5e35-5841-4510-9e95-cc45caeafc79,2024-01-07,Instagram,Share,Neutral


In [45]:
  #Step 3b: Merge the 2 sets earlier to Demographics
merged_dataset= pd.merge(transaction_interaction, demographics, on="CustomerID", how="inner")
#This is an inner join of all datasets

print(f"Total Rows: {len(merged_dataset)}")


duplicate_customers = transactions[transactions['CustomerID'].duplicated()]['CustomerID'].nunique()
print(f"Unique customers with duplicating values: {duplicate_customers}")
#It's normal for customers to have multiple transactions so this won't be removed



Total Rows: 3030
Unique customers with duplicating values: 815


In [46]:
#Step 4: Preprocess Age

#   Step 4a: OUTLIERS


merged_dataset['Age'] = pd.to_numeric(merged_dataset['Age'], errors='coerce')


#     Count outliers before replacement
outlier_count_before = merged_dataset[(merged_dataset['Age'] < 0) | (merged_dataset['Age'] > 99)].shape[0]
print("Age Outliers before:", outlier_count_before)
print("Missing Values before: ", merged_dataset['Age'].isna().sum())

#     Compute mean ignoring outliers
mean_age = merged_dataset[(merged_dataset['Age'] >= 0) & (merged_dataset['Age'] <= 99)]['Age'].mean()

#      Replace outliers with mean
merged_dataset['Age'] = merged_dataset['Age'].apply(
    lambda x: mean_age if (x < 0 or x > 99) else x
)

#      Count outliers after replacement
outlier_count_after = merged_dataset[(merged_dataset['Age'] < 0) | (merged_dataset['Age'] > 99)].shape[0]
print("Age outliers after:", outlier_count_after)

# Step 4b: MISSING VALUES
#     Impute Mean on missing values
merged_dataset["Age"] = merged_dataset["Age"].fillna(mean_age)
print("Missing Values after: ", merged_dataset['Age'].isna().sum())

#  Step 4c: ADD Z SCORE VALUES
merged_dataset["Age_zscore"] = stats.zscore(merged_dataset["Age"])
merged_dataset[["Age_zscore", "Age"]].head()


Age Outliers before: 67
Missing Values before:  266
Age outliers after: 0
Missing Values after:  0


,Age_zscore,Age
0,0.071647,46.000000
1,-0.337167,40.000000
2,-0.269031,41.000000
3,0.000000,44.948461
4,1.706906,70.000000


In [47]:
#Step 5: Preprocess Interaction Date

# Step 5a: Find Invalid Dates
merged_dataset["InteractionDate"] = pd.to_datetime(
    merged_dataset["InteractionDate"], errors="coerce"
)

invalid_dates = merged_dataset["InteractionDate"].isna().sum()
print("Invalid InteractionDate values:", invalid_dates)

# Step 5b: Parse Years to YYYY
merged_dataset["InteractionYear"] = merged_dataset["InteractionDate"].dt.year

# Step 5c: Impute Mode on missing values, fallback to most frequent year
year_counts = merged_dataset["InteractionYear"].value_counts()
print("Year frequency:\n", year_counts)

if not year_counts.empty:
    # pick the most frequent year (e.g., 2023 or 2024)
    fallback_year = year_counts.idxmax()
else:
    # in case the column is all missing
    fallback_year = 2023

merged_dataset["InteractionYear"] = merged_dataset["InteractionYear"].fillna(fallback_year)

print("Fallback year used:", fallback_year)
print("Unique Interaction Years after cleaning:", merged_dataset["InteractionYear"].unique())


Invalid InteractionDate values: 106
Year frequency:
 InteractionYear
2023.0    1512
2024.0    1412
Name: count, dtype: int64
Fallback year used: 2023.0
Unique Interaction Years after cleaning: [2024. 2023.]


In [48]:
#Step 5-ish, Preprocess transaction Dates

merged_dataset["TransactionDate"] = pd.to_datetime(merged_dataset["TransactionDate"], errors="coerce")

invalid_dates = merged_dataset["TransactionDate"].isna().sum()
print("Invalid TransactionDate values:", invalid_dates)

merged_dataset["TransactionYear"] = merged_dataset["TransactionDate"].dt.year

year_counts = merged_dataset["TransactionYear"].value_counts()
print("Year frequency:\n", year_counts)

#Find Mode
mode = merged_dataset["TransactionYear"].mode()[0]
merged_dataset["TransactionYear"] = merged_dataset["TransactionYear"].fillna(mode)

print("Fallback year used:", mode)
print("Unique Transaction Years after cleaning:", merged_dataset["TransactionYear"].unique())





Invalid TransactionDate values: 102
Year frequency:
 TransactionYear
2023.0    1433
2022.0     764
2024.0     731
Name: count, dtype: int64
Fallback year used: 2023.0
Unique Transaction Years after cleaning: [2022. 2024. 2023.]


In [49]:
#Step 6: Preprocess Platform

# Step 6a: Find Invalid Values (missing or empty strings)
invalid_platforms = merged_dataset[
    merged_dataset["Platform"].isna() | (merged_dataset["Platform"].str.strip() == "")
]
print("Invalid Platform count:", invalid_platforms.shape[0])

# Step 6b: Impute mode on missing/invalid values
# Find the most frequent platform (mode) excluding blanks/NaN
platform_mode = merged_dataset.loc[
    merged_dataset["Platform"].notna() & (merged_dataset["Platform"].str.strip() != ""),
    "Platform"
].mode()

if not platform_mode.empty:
    platform_mode = platform_mode[0]
else:
    platform_mode = "Unknown"  # fallback if column is completely empty

# Replace blanks with NaN first
merged_dataset["Platform"] = merged_dataset["Platform"].replace("", None)

# Fill missing with mode (or fallback)
merged_dataset["Platform"] = merged_dataset["Platform"].fillna(platform_mode)

print("Unique Platforms after cleaning:", merged_dataset["Platform"].nunique())


Invalid Platform count: 238
Unique Platforms after cleaning: 3


In [50]:
#Step 7: Preprocess Interaction Type

# Step 7a: Find Invalid Values (missing or empty strings)
invalid_interactions_before = merged_dataset[
    merged_dataset["InteractionType"].isna() | (merged_dataset["InteractionType"].str.strip() == "")
]

print("Invalid InteractionType counts before cleaning:", invalid_interactions_before.shape[0])
print("Unique Interaction Types before cleaning:", merged_dataset["InteractionType"].nunique())

# Step 7b: Impute mode on missing/invalid values
interaction_mode = merged_dataset.loc[
    merged_dataset["InteractionType"].notna() & (merged_dataset["InteractionType"].str.strip() != ""),
    "InteractionType"
].mode()

if not interaction_mode.empty:
    interaction_mode = interaction_mode[0]
else:
    interaction_mode = "Unknown"  # fallback if column has no valid entries

# Replace blanks with NaN first
merged_dataset["InteractionType"] = merged_dataset["InteractionType"].replace("", None)

# Fill missing/invalid with mode (or "Unknown")
merged_dataset["InteractionType"] = merged_dataset["InteractionType"].fillna(interaction_mode)

# Step 7c: Check again after cleaning
invalid_interactions_after = merged_dataset[
    merged_dataset["InteractionType"].isna() | (merged_dataset["InteractionType"].str.strip() == "")
]

print("Invalid InteractionType counts after cleaning:", invalid_interactions_after.shape[0])
print("Unique Interaction Types after cleaning:", merged_dataset["InteractionType"].nunique())
print("InteractionType value counts after cleaning:\n", merged_dataset["InteractionType"].value_counts())

Invalid InteractionType counts before cleaning: 0
Unique Interaction Types before cleaning: 3
Invalid InteractionType counts after cleaning: 0
Unique Interaction Types after cleaning: 3
InteractionType value counts after cleaning:
 InteractionType
Share      1040
Like       1002
Comment     988
Name: count, dtype: int64


In [51]:
#Step 8: Preprocess Sentiment

# Step 8a: Find Invalid Values (missing or empty strings)
invalid_sentiments_before = merged_dataset[
    merged_dataset["Sentiment"].isna() | (merged_dataset["Sentiment"].str.strip() == "")
]

print("Invalid Sentiment counts before cleaning:", invalid_sentiments_before.shape[0])
print("Unique Sentiments before cleaning:", merged_dataset["Sentiment"].nunique())

# Step 8b: Impute mode on missing/invalid values
sentiment_mode = merged_dataset.loc[
    merged_dataset["Sentiment"].notna() & (merged_dataset["Sentiment"].str.strip() != ""),
    "Sentiment"
].mode()

if not sentiment_mode.empty:
    sentiment_mode = sentiment_mode[0]
else:
    sentiment_mode = "Unknown"  # fallback if no valid sentiments exist

# Replace blanks with NaN first
merged_dataset["Sentiment"] = merged_dataset["Sentiment"].replace("", None)

# Fill missing/invalid with mode (or "Unknown")
merged_dataset["Sentiment"] = merged_dataset["Sentiment"].fillna(sentiment_mode)

# Step 8c: Check again after cleaning
invalid_sentiments_after = merged_dataset[
    merged_dataset["Sentiment"].isna() | (merged_dataset["Sentiment"].str.strip() == "")
]

print("Invalid Sentiment counts after cleaning:", invalid_sentiments_after.shape[0])
print("Unique Sentiments after cleaning:", merged_dataset["Sentiment"].nunique())
print("Sentiment value counts after cleaning:\n", merged_dataset["Sentiment"].value_counts())

Invalid Sentiment counts before cleaning: 304
Unique Sentiments before cleaning: 5
Invalid Sentiment counts after cleaning: 0
Unique Sentiments after cleaning: 5
Sentiment value counts after cleaning:
 Sentiment
Positive         1239
Neutral           873
Negative          839
Very Positive      43
Very Negative      36
Name: count, dtype: int64


In [52]:
#Step 10: Preprocess AMount

# Step 10a: Find Outliers
# Convert to numeric, forcing invalids to NaN
merged_dataset["Amount"] = pd.to_numeric(merged_dataset["Amount"], errors="coerce")

# Define outlier condition (e.g., negative values or extreme values beyond z-score threshold)
outliers_before = merged_dataset[
    (merged_dataset["Amount"] < 0) | (merged_dataset["Amount"] > merged_dataset["Amount"].quantile(0.99))
]
print("Outliers in Amount before cleaning:", outliers_before.shape[0])
print("Missing Amount values before cleaning:", merged_dataset["Amount"].isna().sum())

# Step 10b: Impute Mean on missing values
amount_mean = merged_dataset[merged_dataset["Amount"] >= 0]["Amount"].mean()
amount_q99 = merged_dataset["Amount"].quantile(0.99)  # freeze threshold

# Replace negative/outlier values with mean
merged_dataset["Amount"] = merged_dataset["Amount"].apply(
    lambda x: amount_mean if (pd.isna(x) or x < 0 or x > amount_q99) else x
)

print("Missing Amount values after cleaning:", merged_dataset["Amount"].isna().sum())
print("Outliers in Amount after cleaning:", merged_dataset[
    (merged_dataset["Amount"] < 0) | (merged_dataset["Amount"] > amount_q99)
].shape[0])


# Step 10c: Add Z-score column
merged_dataset["Amount_zscore"] = stats.zscore(merged_dataset["Amount"])
print("Amount with Z-score column added.")
print(merged_dataset[["Amount", "Amount_zscore"]].head())


Outliers in Amount before cleaning: 69
Missing Amount values before cleaning: 319
Missing Amount values after cleaning: 0
Outliers in Amount after cleaning: 0
Amount with Z-score column added.
   Amount  Amount_zscore
0  563.99       0.244511
1  254.44      -0.884530
2  590.52       0.341276
3  399.70      -0.354714
4  296.99      -0.729335


In [53]:
#Step 11: Preprocess Product Category

# Step 11a: Find Invalid Values (missing or empty strings)
invalid_categories = merged_dataset[
    merged_dataset["ProductCategory"].isna() | (merged_dataset["ProductCategory"].str.strip() == "")
]

print("Invalid ProductCategory counts before cleaning:", invalid_categories.shape[0])
print("Unique ProductCategories before cleaning:", merged_dataset["ProductCategory"].nunique())

# Step 11b: Impute mode on missing/invalid values
# Find the mode (most common category), ignoring blanks/NaN
category_mode = merged_dataset.loc[
    merged_dataset["ProductCategory"].notna() & (merged_dataset["ProductCategory"].str.strip() != ""),
    "ProductCategory"
].mode()

if not category_mode.empty:
    category_mode = category_mode[0]
else:
    category_mode = "Unknown"  # fallback if column is completely empty

# Replace blanks with NaN first
merged_dataset["ProductCategory"] = merged_dataset["ProductCategory"].replace("", None)

# Fill missing/invalid with mode or fallback
merged_dataset["ProductCategory"] = merged_dataset["ProductCategory"].fillna(category_mode)

print("Invalid ProductCategory counts after cleaning:",
      merged_dataset["ProductCategory"].isna().sum() + (merged_dataset["ProductCategory"].str.strip() == "").sum())
print("Unique ProductCategories after cleaning:", merged_dataset["ProductCategory"].nunique())



Invalid ProductCategory counts before cleaning: 271
Unique ProductCategories before cleaning: 5
Invalid ProductCategory counts after cleaning: 0
Unique ProductCategories after cleaning: 5


In [54]:
#Step 12: Preprocess Payment Method

# Step 12a: Find Invalid Values (missing or empty strings)
print("Before Cleaning:")
print("Invalid PaymentMethod counts:",
      merged_dataset["PaymentMethod"].isna().sum() +
      (merged_dataset["PaymentMethod"].astype(str).str.strip() == "").sum())
print("Unique PaymentMethods before cleaning:", merged_dataset["PaymentMethod"].nunique())

# Step 12b: Impute mode on missing/invalid values
# Find the most frequent PaymentMethod (mode) excluding blanks/NaN
payment_mode = merged_dataset.loc[
    merged_dataset["PaymentMethod"].notna() &
    (merged_dataset["PaymentMethod"].astype(str).str.strip() != ""),
    "PaymentMethod"
].mode()

if not payment_mode.empty:
    payment_mode = payment_mode[0]
else:
    payment_mode = "Unknown"  # fallback if column is empty

# Replace blanks with NaN first
merged_dataset["PaymentMethod"] = merged_dataset["PaymentMethod"].replace("", None)

# Fill missing/invalid with mode (or "Unknown")
merged_dataset["PaymentMethod"] = merged_dataset["PaymentMethod"].fillna(payment_mode)

print("\nAfter Cleaning:")
print("Invalid PaymentMethod counts:",
      merged_dataset["PaymentMethod"].isna().sum() +
      (merged_dataset["PaymentMethod"].astype(str).str.strip() == "").sum())
print("Unique PaymentMethods after cleaning:", merged_dataset["PaymentMethod"].nunique())


Before Cleaning:
Invalid PaymentMethod counts: 0
Unique PaymentMethods before cleaning: 4

After Cleaning:
Invalid PaymentMethod counts: 0
Unique PaymentMethods after cleaning: 4


In [55]:
#Step 13: Preprocess Gender

#  Step 13a: Find Invalid Values (e.g., anything not Male/Female)

valid_genders = ["Male", "Female"]
invalid_genders = merged_dataset[~merged_dataset["Gender"].isin(valid_genders)]
print("Invalid Gender values:\n", invalid_genders["Gender"].unique())

# Step 13b: Impute mode on missing/invalid values
gender_mode = merged_dataset.loc[merged_dataset["Gender"].isin(valid_genders), "Gender"].mode()[0]
merged_dataset["Gender"] = merged_dataset["Gender"].where(merged_dataset["Gender"].isin(valid_genders))
merged_dataset["Gender"] = merged_dataset["Gender"].fillna(gender_mode)

print("Unique Gender values after cleaning:", merged_dataset["Gender"].unique())

Invalid Gender values:
 []
Unique Gender values after cleaning: ['Female' 'Male']


In [56]:
#Step 14: Preprocess Location

# Step 14a: Find Invalid Values (empty strings or NaN)
invalid_locations = merged_dataset[merged_dataset["Location"].isna() | (merged_dataset["Location"].str.strip() == "")]
print("Invalid Locations count:", invalid_locations.shape[0])

# Step 14b: Impute mode on missing/invalid values
merged_dataset["Location"] = merged_dataset["Location"].str.strip().str.title()
# Step 14c: Impute mode on missing/invalid values
location_mode = merged_dataset.loc[
    merged_dataset["Location"].notna() & (merged_dataset["Location"] != ""),
    "Location"
].mode()

if not location_mode.empty:
    location_mode = location_mode[0]
else:
    location_mode = "Unknown"

merged_dataset["Location"] = merged_dataset["Location"].replace("", None)
merged_dataset["Location"] = merged_dataset["Location"].fillna(location_mode)

print("Unique Locations after cleaning:", merged_dataset["Location"].nunique())

Invalid Locations count: 0
Unique Locations after cleaning: 1128


In [65]:
#Step 15: Preprocess Income Level

# Step 15a: Find Invalid Values
valid_income = ["High", "Medium", "Low"]
invalid_income = merged_dataset[~merged_dataset["IncomeLevel"].isin(valid_income)]
print("Invalid IncomeLevel values:", invalid_income["IncomeLevel"].unique().sum())

# Step 15b: Impute mode on missing/invalid values
income_mode = merged_dataset.loc[merged_dataset["IncomeLevel"].isin(valid_income), "IncomeLevel"].mode()

if not income_mode.empty:
    income_mode = income_mode[0]
else:
    income_mode = "Medium"  # fallback if dataset is empty

merged_dataset["IncomeLevel"] = merged_dataset["IncomeLevel"].where(
    merged_dataset["IncomeLevel"].isin(valid_income)
)
merged_dataset["IncomeLevel"] = merged_dataset["IncomeLevel"].fillna(income_mode)

print("Unique Income Levels after cleaning:", merged_dataset["IncomeLevel"].unique())

Invalid IncomeLevel values: 0
Unique Income Levels after cleaning: ['Low' 'High' 'Medium']


In [58]:
#Step 16: Preprocess Sign up Date

# Step 16a: Find Invalid Dates
merged_dataset["SignupDate"] = pd.to_datetime(merged_dataset["SignupDate"], errors="coerce")
invalid_dates = merged_dataset["SignupDate"].isna().sum()
print("Invalid SignupDate values:", invalid_dates)

# Step 16b: Parse Years to YYYY (extract only year)
merged_dataset["SignUpYear"] = merged_dataset["SignupDate"].dt.year

# Step 16c: Impute Mode on missing values
year_mode = merged_dataset["SignUpYear"].mode()[0]
merged_dataset["SignUpYear"] = merged_dataset["SignUpYear"].fillna(year_mode)

print("Unique SignUp Years after cleaning:", merged_dataset["SignUpYear"].unique())
print(merged_dataset["SignUpYear"].isnull().sum())

Invalid SignupDate values: 132
Unique SignUp Years after cleaning: [2022. 2021. 2020. 2023. 2024. 2019.]
0


In [59]:
print(merged_dataset.columns.tolist())

['CustomerID', 'TransactionID', 'TransactionDate', 'Amount', 'ProductCategory', 'PaymentMethod', 'InteractionID', 'InteractionDate', 'Platform', 'InteractionType', 'Sentiment', 'Age', 'Gender', 'Location', 'IncomeLevel', 'SignupDate', 'Age_zscore', 'InteractionYear', 'TransactionYear', 'Amount_zscore', 'SignUpYear']


In [60]:
merged_dataset = merged_dataset.drop(columns=["SignupDate"])
merged_dataset = merged_dataset.drop(columns=["TransactionDate"])
merged_dataset = merged_dataset.drop(columns=["InteractionDate"])

In [61]:
merged_dataset.to_csv("Cleaned_Data.csv", index=False)

In [62]:
from google.colab import files
#files.download("Cleaned_Data.csv")

#Exploratory Data Analysis

#Data Inspection


In [68]:

print("----------------------HEAD---------------------------------")
print(merged_dataset.head())
print("--------------------INFO---------------------------------------")
print(merged_dataset.info())
print("--------------------DESCRIBE----------------------------------------")
print(merged_dataset.describe())
print("------------------------NULLS------------------------------------")
print(merged_dataset.isnull().sum())
print("-------------------------SHAPE-----------------------------------")
print(merged_dataset.shape)
print("--------------------FREQUENCY DISTRIBUTION----------------------------------")
for col in ["ProductCategory", "PaymentMethod", "Platform",
            "InteractionType", "Sentiment", "Gender",
            "Location", "IncomeLevel"]:
    print(f"\nFrequency distribution for {col}:")
    print(merged_dataset[col].value_counts())
    print("-----------")



----------------------HEAD---------------------------------
                             CustomerID                         TransactionID  \
0  9223891b-73ff-4d5c-b8ae-13ece82ee28b  f79588dd-3db9-4ffa-97f8-7de0e64259f1   
1  9243eebc-938f-480c-8564-16d503d250de  401c0fc9-60df-4455-ad78-67c132f9897d   
2  6e3e8eb8-bc0f-4ffe-9f74-5d5efec9502f  2034aebc-8280-4254-a667-92bcd1c2be4f   
3  958e5c8d-48ca-42dd-bb71-a766a374233a  fb24e098-3ab9-40a2-bcc3-b8ebb23f549a   
4  39c6e7d2-6c4b-44c0-8961-ddc1ecbdb0c6  833b026a-7c02-4101-832d-62c07569b0f6   

   Amount ProductCategory  PaymentMethod  \
0  563.99        Clothing     Debit Card   
1  254.44      Automotive         PayPal   
2  590.52   Home & Garden  Bank Transfer   
3  399.70   Home & Garden         PayPal   
4  296.99        Clothing         PayPal   

                          InteractionID   Platform InteractionType Sentiment  \
0  df9b66ed-008c-4484-9e9a-83ab83206287    Twitter            Like   Neutral   
1  dcbde36b-7cfb-4220-93da-8

Mode: [500.86892348]
